# 영상 분할

영상 분할(Image segmentation)은 입력 영상에서 관심 있는 영역을 분리하는 과정입니다. 영상 분할은 영상 분석, 물체 인식, 추적 등 대부분의 영상 처리 응용에서 필수적인 단계입니다. 

## 이미지 필터 

이미지 필터 커널(Filter Kernel, 줄여서 필터라고도 함) 행렬의 특성에 따라 원본 이미지로부터 특성이 강조된 이미지를 얻을 수 있습니다. 어떤 필터를 사용하느냐에 따라 그 결과는 전혀 다릅니다. 

## 합성곱

합성곱(Convolution)은 필터를 적용할 수 있는 영역의 주목화소와 주위 화소에 커널 배열을 곱한 후 모든 항목을 더하는 것을 말합니다. 

In [1]:
# 합성곱 함수 구현 및 적용
import numpy as np
import cv2

def Conv2D(img, kernel=None, padding="valid"):
    if kernel is not None :
        sy, sx = int(len(kernel)/2), int(len(kernel[0])/2) 
        h, w = img.shape[0:2]
        if padding=="same": # zero padding
            new_shape = (h+2*sy, w+2*sx)
            img_out = np.zeros(new_shape, dtype=np.uint8)
        elif padding=="valid": # 처리 못하는 바깥 화소를 처리 안함
            img_out = np.zeros(img.shape, dtype=np.uint8)

        height, width = img_out.shape
        for y in range(sy, height-sy):
            for x in range(sx, width-sx):
                roi = img[y-sy:y+sy+1, x-sx:x+sx+1]
                y_, x_ = roi.shape
                filtered = roi * kernel[:y_, :x_]
                pixel = np.abs(np.sum(filtered))
                img_out[y, x] = np.uint8(pixel)
                
        return img_out[sy:-sy, sx:-sx]
    else:
        print("Mask array not found!")
        
img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)
kernel = np.array([[0,0,0],[0,1,-1],[0,0,0]])
output = Conv2D(img, kernel=kernel, padding="same")
cv2.imshow("House", img)
cv2.imshow("Conv2D", output)
cv2.waitKey()
cv2.destroyAllWindows()

### cv2.filter2D(src, ddepth, kernel, anchor=None, delta=None, borderType=None) -> dst

OpenCV의 합성곱 함수는 cv2.filter2D()입니다. 이 함수를 이용하면 쉽게 필터를 적용할 수 있습니다. 

- src : 입력 이미지입니다. 
- ddepth : 출력 영상의 데이터 타입(cv2.CV_XXX 형식)을 지정합니다. -1을 지정하면 src와 같은 타입의 dst 영상을 생성합니다. 
- kernel : 필터 커널 행렬입니다. 실수형입니다.
- anchor : 고정점 위치입니다. (-1, -1)이면 필터 중앙을 고정점으로 사용합니다.
- delta : 추가적으로 더할 값입니다.
- borderType : 가장자리 픽셀 확장 방식을 지정합니다.

반환값 dst는 출력 영상입니다.

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)

kernel = np.array([[0,0,0],[0,1,-1],[0,0,0]])
output = cv2.filter2D(img,-1,kernel, borderType=cv2.BORDER_REFLECT)

cv2.imshow("House", img)
cv2.imshow("filter2D", output)
cv2.waitKey()
cv2.destroyAllWindows()

## 엣지 검출 필터 

이미지의 특징을 찾기 위해 전처리하는 것 중 하나가 경계선 영역을 찾는 것입니다. 이미지의 경계선 또는 윤곽선을 엣지(edge)라고 부르는데, 엣지를 추출하기 위해 차분 필터, 로버츠 필터, 프리위트 필터, 소벨 필터, 라플라시안 필터 등이 있습니다. 

### 차분 필터 

차분(Differential) 필터는 현재 화소와 오른쪽 및 아래 이웃 화소의 차이를 이용해서 엣지를 찾습니다. 

### 로버츠 필터 

로버츠(Rovert) 필터는 대각선 방향의 이웃 화소의 차이를 이용해서 엣지를 찾습니다. 그래서 대각선 경계를 찾는데 유리합니다. 

### 라플라시안 필터

라플라시안(Laplace) 필터는 현재 주목화소에 가중치를 더 많이 부여합니다. 중앙값에 가중치를 더 주면 테두리 성분이 더 부각되게 나타나는 결과를 얻을 수 있습니다. 

### 소벨 필터

소벨(Sobel) 필터는 1968년 스탠포드 인공지능 연구소에서 어윈 소벨이 고안해낸 엣지 검출 알고리즘으로 3X3 크기의 행렬을 사용하여 연산했을 경우 중심을 기준으로 각 방향의 앞뒤 값을 비교하여 변화량을 검출하는 알고리즘입니다. 

In [3]:
# 차분 필터 함수 구현 및 적용
import numpy as np
import cv2

def diff_filter(img):
    height, width = img.shape
    img_ = np.zeros(img.shape, dtype=np.uint8)
    for y in range(height):
        for x in range(width):
            try :
                H = 0*img[y-1, x-1] +  0*img[y-1, x] +  0*img[y-1, x+1] \
                  + 0*img[y  , x-1] +  1*img[y  , x] + -1*img[y  , x+1] \
                  + 0*img[y+1, x-1] +  0*img[y+1, x] +  0*img[y+1, x+1] 
                V = 0*img[y-1, x-1] +  0*img[y-1, x] +  0*img[y-1, x+1] \
                  + 0*img[y  , x-1] +  1*img[y  , x] +  0*img[y  , x+1] \
                  + 0*img[y+1, x-1] + -1*img[y+1, x] +  0*img[y+1, x+1] 
                img_[y,x] = np.sqrt(H**2 + V**2)
            except:
                pass
    return img_

img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Origin", img)
cv2.imshow("Diff Filter", diff_filter(img))
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
# filter2D() 함수를 이용한 차분필터 적용
import numpy as np
import cv2

img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)
x_kernel = np.array([[-1, 1]])
y_kernel = np.array([[-1],[1]])

x_edge = cv2.filter2D(img, -1, x_kernel)
y_edge = cv2.filter2D(img, -1, y_kernel)

cv2.imshow("Origin", img)
cv2.imshow("Diff Filter X Y", np.c_[x_edge, y_edge])
cv2.imshow("Diff Filter", x_edge+y_edge)
cv2.waitKey()
cv2.destroyAllWindows()

In [5]:
# 로버츠 필터 함수 구현 및 적용 
import numpy as np
import cv2

def roverts_filter(img):
    height, width = img.shape
    img_ = np.zeros(img.shape, dtype=np.uint8)
    for y in range(height):
        for x in range(width):
            try :
                H =  0*img[y-1, x-1] +  0*img[y-1, x] +  0*img[y-1, x+1] \
                  +  0*img[y  , x-1] +  1*img[y  , x] +  0*img[y  , x+1] \
                  +  0*img[y+1, x-1] +  0*img[y+1, x] + -1*img[y+1, x+1] 
                
                V =  0*img[y-1, x-1] +  0*img[y-1, x] +  0*img[y-1, x+1] \
                  +  0*img[y  , x-1] +  0*img[y  , x] +  1*img[y  , x+1] \
                  +  0*img[y+1, x-1] + -1*img[y+1, x] +  0*img[y+1, x+1] 
                img_[y,x] = np.sqrt(H**2 + V**2)
            except:
                pass
    return img_

img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Origin", img)
cv2.imshow("Roverts Filter", diff_filter(img))
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
# filter2D() 함수를 이용한 로버츠필터 적용
import numpy as np
import cv2

img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)
x_kernel = np.array([[1,0],[0, -1]])
y_kernel = np.array([[0,1],[-1,0]])

x_edge = cv2.filter2D(img, -1, x_kernel)
y_edge = cv2.filter2D(img, -1, y_kernel)

cv2.imshow("Origin", img)
cv2.imshow("Roverts Filter X Y", np.c_[x_edge, y_edge])
cv2.imshow("Roverts Filter", x_edge+y_edge)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
# 라플라시안 필터 함수 구현 및 적용
import numpy as np
import cv2

def laplacian_filter(img):
    img_ = np.zeros(img.shape, dtype=np.uint8)
    height, width = img.shape[:2]
    for y in range(height):
        for x in range(width):
            try :
                T =  0*img[y-1, x-1] + -1*img[y-1, x] +  0*img[y-1, x+1] \
                  + -1*img[y  , x-1] +  4*img[y  , x] + -1*img[y  , x+1] \
                  +  0*img[y+1, x-1] + -1*img[y+1, x] +  0*img[y+1, x+1] 

                img_[y,x] = np.abs(T)
            except:
                pass
    return img_

img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Origin", img)
cv2.imshow("Laplacian Filter", laplacian_filter(img))
cv2.waitKey()
cv2.destroyAllWindows()

### cv2.Laplacian(src, ddepth, dst, ksize, scale, delta) -> dst

- src : 입력 이미지입니다.
- ddepth : 출력 이미지의 깊이입니다. ddepth=-1 이면 출력 이미지는 소스와 동일한 깊이를 갖습니다.
- dst : src와 동일한 크기 및 채널 수의 출력 이미지입니다.
- ksize : 필터 커널의 크기(1, 3, 5, 7 중 하나)입니다. 
- scale : 미분 값에 대한 계수입니다. 
- delta : dst에 저장하기 전에 결과에 더해져야 하는 값입니다. 

In [9]:
import numpy as np
import cv2

img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Origin", img)
cv2.imshow("cv2.Laplacian", cv2.Laplacian(img, -1))
cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
# 소벨 필터 함수 구현 및 적용 
import numpy as np
import cv2


def sobel_filter(img):
    img_ = np.zeros(img.shape, dtype=np.uint8)
    height, width = img.shape[:2]
    for y in range(height):
        for x in range(width):
            try :
                H =  1*img[y-1, x-1] +  0*img[y-1, x] + -1*img[y-1, x+1] \
                  +  2*img[y  , x-1] +  0*img[y  , x] + -2*img[y  , x+1] \
                  +  1*img[y+1, x-1] +  0*img[y+1, x] + -1*img[y+1, x+1] 
                
                V =  1*img[y-1, x-1] +  2*img[y-1, x] +  1*img[y-1, x+1] \
                  +  0*img[y  , x-1] +  0*img[y  , x] +  0*img[y  , x+1] \
                  + -1*img[y+1, x-1] + -2*img[y+1, x] + -1*img[y+1, x+1] 

                img_[y,x] = (np.abs(H) + np.abs(V))/2
            except:
                pass
    return img_

img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Origin", img)
cv2.imshow("Sobel Filter", sobel_filter(img))
cv2.waitKey()
cv2.destroyAllWindows()

### cv2.Sobel(src, ddepth, dx, dy, dst, ksize, scale, delta) -> dst

- src : 입력 이미지입니다.
- ddepth : 출력 이미지의 깊이입니다. ddepth=-1 이면 출력 이미지는 소스와 동일한 깊이를 갖습니다.
- dx, dy : x, y의 미분 차수입니다. x 방향으로 1차 미분은 1, 0이며, y 방향으로 1차 미분은 0, 1입니다. 0, 1, 2 중 하나여야 하며, dx, dy 둘 다 0일 수는 없습니다. 
- dst : src와 동일한 크기 및 채널 수의 출력 이미지입니다.
- ksize : 필터 커널의 크기(1, 3, 5, 7 중 하나)입니다. 
- scale : 미분 값에 대한 계수입니다. 
- delta : dst에 저장하기 전에 결과에 더해져야 하는 값입니다. 

In [11]:
import numpy as np
import cv2

img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)

x_edge = cv2.Sobel(img, -1, 1, 0, ksize=3)
y_edge = cv2.Sobel(img, -1, 0, 1, ksize=3)

cv2.imshow("Origin", img)
cv2.imshow("Sobel Filter X Y", np.c_[x_edge, y_edge])
cv2.imshow("Sobel Filter", x_edge+y_edge)
cv2.waitKey()
cv2.destroyAllWindows()

## 캐니 엣지

캐니 엣지는 1986년 존 캐니가 제안한 알고리즘입니다. 캐니 엣지 알고리즘은 다음 단계에 걸쳐 엣지를 검출합니다. 

1. 노이즈 제거 - 가우시안 필터를 이용해 이미지의 노이즈를 제거합니다.
2. 소벨 필터로 엣지 찾기 - 수평 및 수직 방향 모두에서 소벨 필터 커널로 필터링되어 1차 도함수를 얻습니다. 
3. 기울기 크기 임계값 또는 하한 컷오프 억제 적용 - 엣지 구성에 불필요한 픽셀을 제거하기 위한 스캔이 수행됩니다.
4. 히스테리시스 임계값 - 모든 엣지가 실제로 엣지인지 아닌지 결정합니다.

### cv2.Canny(img, minVal, maxVal, apertureSize, L2gradient)

- img : 입력 이미지입니다. 
- minVal : 캐니 엣지 알고리즘이 마지막 단계인 히스테리시스를 수행하기 위한 임계값 1입니다. 
- maxVal : 캐니 엣지 알고리즘이 마지막 단계인 히스테리시스를 수행하기 위한 임계값 2입니다. 
- apertureSize : 소벨 필터의 크기입니다. 3, 5, 7만 지정 가능합니다. 
- L2gradient : 기울기 크기를 찾기 위한 방정식을 지정합니다. True, False에 따른 방정식이 다릅니다.

In [12]:
import numpy as np
import cv2

img = cv2.imread("./data/stitch_image1.jpg", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Origin", img)
cv2.imshow("cv2.Laplacian", cv2.Laplacian(img, -1))
cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
import cv2
import numpy as np

src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

edges1 = cv2.Canny(src, 50, 100)
edges2 = cv2.Canny(src, 50, 200)
 
cv2.imshow('edges1',  edges1)
cv2.imshow('edges2',  edges2)
cv2.waitKey()
cv2.destroyAllWindows()